In [0]:
import tensorflow as tf

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from tensorflow.examples.tutorials.mnist import  input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

W0730 14:29:31.251749 139729501616000 deprecation.py:323] From <ipython-input-3-27afc53753d9>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0730 14:29:31.253098 139729501616000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0730 14:29:31.254861 139729501616000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0730 14:29:32.002633 139729501616000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0730 14:29:32.005130 139729501616000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0730 14:29:32.623369 139729501616000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
mnist.train.images.shape,mnist.train.labels.shape

((55000, 784), (55000, 10))

In [0]:
input_width=28
input_height=28
input_channels=1
input_pixels=784

n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2
n_hidden=1024
n_out=10

input_size_to_hidden=(input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k))* n_conv2

In [0]:
weights={
    'wc1':tf.Variable(tf.random_normal(shape=(conv1_k,conv1_k,input_channels, n_conv1))),  # weight matrix for layer 1
    'wc2':tf.Variable(tf.random_normal(shape=(conv2_k,conv2_k,n_conv1, n_conv2))),    # weight matrix for layer 2
   'wh1':tf.Variable(tf.random_normal(shape=(input_size_to_hidden, n_hidden))),    # weight matrix for layer 2'
    'wo':tf.Variable(tf.random_normal(shape=(n_hidden,n_out))) # weight matrix for layer 3
}
biases={
    'bc1':tf.Variable(tf.random_normal([n_conv1])),
     'bc2':tf.Variable(tf.random_normal([n_conv2])),
    'bh1':tf.Variable(tf.random_normal([n_hidden])),         # biases for layer 1
    'bo':tf.Variable(tf.random_normal([n_out])),      # biases for layer 1
}

In [0]:
def conv(x,weights,bias,strides=1):
    out=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,strides,strides,1])
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [0]:
def cnn(x,weights,biases,keep_prob):
    x=tf.reshape(x,shape=[-1,input_width, input_height, input_channels])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool=maxpooling(conv1,max_pool1_k)
    
    conv2=conv(conv1_pool, weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool=maxpooling(conv2,max_pool2_k)
    
    hidden_input=tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])
    hidden_output_before_activation=tf.add(tf.matmul(hidden_input,weights['wh1']), biases['bh1'])
    
    hidden_output_before_dropout=tf.nn.relu(hidden_output_before_activation)
    hidden_output=tf.nn.dropout( hidden_output_before_dropout, keep_prob)
    
    output=tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output
    

In [9]:
x=tf.placeholder('float',[None,input_pixels])      #since no. of input instance is different for training and testing
y=tf.placeholder(tf.int32,[None,n_out])
keep_prob=tf.placeholder('float')
pred=cnn(x,weights,biases,keep_prob)

W0730 14:29:33.005907 139729501616000 deprecation.py:506] From <ipython-input-8-9ab4005eebc2>:13: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y)) #logits is because of multiple classes 

optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)
sess=tf.Session()
sess.run(tf.global_variables_initializer())
batch_size=100
for i in range(25):
    num_batches=mnist.train.num_examples//batch_size
    total_cost=0
    for j in range(num_batches):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        c,_=sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y, keep_prob:0.8 })
        total_cost+=c
    print(total_cost)    
    


787896.6421284676
35576.514347583055
20872.848658851868
14903.333839154897
11255.679214928474
8660.02426001517
8679.162090844879
6713.049797238945
6819.021234335009
5557.916645219733
4810.24178134071
4709.388835262811
2972.4738087332253
3457.9532778168523
3091.9847892526873
3084.958458912325
3173.997729080649
2770.7447603702767
2442.03072940404
2492.9233363270773
2383.473474850284
2625.566243447038
2155.0809841815935
1592.7668598231346
1604.33840451572


In [11]:
predictions=tf.argmax(pred,1)
true_labels=tf.argmax(y,1)
correct_preds=tf.equal(predictions,true_labels)
correct_ones,labels,correct_preds=sess.run([predictions, true_labels,correct_preds],feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1})
correct_ones,labels,correct_preds

(array([7, 2, 1, ..., 4, 5, 6]),
 array([7, 2, 1, ..., 4, 5, 6]),
 array([ True,  True,  True, ...,  True,  True,  True]))